In [1]:
import os
from experiment_setup import setups

In [2]:
# download the dataset, the links can be fund in the README
dataset_path = "../datasets/coveo_ecommerce"
model_path = "../trained_models"

## Run the preprocess script, specific to the dataset you chose

- The preprocessing script in general, executes the following steps:
    - Loads the raw data, with correct types
    - Creates the sessions
    - Removes duplicated items. An item is considered as a duplicate if the preceding (based on time) event in the same session contains the exact same item.
    - Performes iterative support filtering
        - Removes sessions with only one event
        - Removes items with less than 5 events
        - Until the size of the dataset changes


In [4]:
%run ../Preprocess/coveo_preproc.py --path $dataset_path

1566074 274797 11365
1464757 173480 11344
1463706 173480 10869
1463649 173423 10869
1463645 173423 10868
1463645 173423 10868
1463645 173423 10868


C:\Users\EVILAB\Documents\RecSys-Project-2024-1\Preprocess\coveo_preproc.py:177: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  dt.datetime.utcfromtimestamp(data.Time.min() / 1000).strftime(
C:\Users\EVILAB\Documents\RecSys-Project-2024-1\Preprocess\coveo_preproc.py:182: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  dt.datetime.utcfromtimestamp(data.Time.max() / 1000).strftime(
C:\Users\EVILAB\Documents\RecSys-Project-2024-1\Preprocess\coveo_preproc.py:177: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent d

                                             Dataset  NumEvents  NumSessions  \
0      coveo_ecommerce\coveo_processed_view_full.tsv    1463645       173423   
1      coveo_ecommerce\coveo_processed_view_test.tsv      52501         7748   
2  coveo_ecommerce\coveo_processed_view_train_ful...    1411113       165673   
3  coveo_ecommerce\coveo_processed_view_train_tr.tsv    1368003       159766   
4  coveo_ecommerce\coveo_processed_view_train_val...      43032         5905   

   NumItems    NumDays                   StartTime  \
0     10868  17.999833  2018-12-08 00:00:11.994000   
1      8230   0.998696  2018-12-25 00:01:50.223000   
2     10868  16.999566  2018-12-08 00:00:11.994000   
3     10868  15.999713  2018-12-08 00:00:11.994000   
4      8014   0.997503  2018-12-24 00:03:10.240000   

                      EndTime  AvgItemViews  MinSessionLength  \
0  2018-12-25 23:59:57.577000    134.674733                 2   
1  2018-12-25 23:59:57.577000      6.379222                 2   

C:\Users\EVILAB\Documents\RecSys-Project-2024-1\Preprocess\coveo_preproc.py:177: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  dt.datetime.utcfromtimestamp(data.Time.min() / 1000).strftime(
C:\Users\EVILAB\Documents\RecSys-Project-2024-1\Preprocess\coveo_preproc.py:182: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  dt.datetime.utcfromtimestamp(data.Time.max() / 1000).strftime(


## Use a specific setup for your dataset

In [3]:
params = setups["coveo"]["params_xe"]

In [4]:
train_path = os.path.join(dataset_path,"coveo_processed_view_train_full.tsv")
test_path = os.path.join(dataset_path,"coveo_processed_view_test.tsv")

In [5]:
def create_gru4rec_tensorflow_script(model_name, train_path, test_path, model_path, loss, final_act, layers, batch_size, dropout_p_hidden, learning_rate, hidden_act, n_epochs, m, decay, initial_accumulator):
        """Creates the train and test script.
        """
        s_train_full = (
                f"python ../GRU4Rec_TensorFlow/gru4rec_BP/main.py --train_path {train_path} "
                f"--test_path {test_path} --checkpoint_dir {model_path}/{model_name} --layer {1} --size {layers} "
                f"--epoch {n_epochs} --lr {learning_rate} --hidden_act {hidden_act} --final_act {final_act} "
                f"--loss {loss} --dropout {dropout_p_hidden} --batch_size {batch_size} "
                f"--initial_accumulator_value {initial_accumulator} --decay {decay}"
        )
        s_test_full = s_train_full + f" --train 0 --test {n_epochs-1} --m {m}"
        return s_train_full, s_test_full

In [6]:
loss = params["loss"]
optim = params["optim"]
const_emb = params["constrained_embedding"]
embed = params["embedding"]
final_act = params["final_act"]
layers = params["layers"]
batch_size = params["batch_size"]
dropout_p_embed = params["dropout_p_embed"]
dropout_p_hidden = params["dropout_p_hidden"]
learning_rate = params["learning_rate"]
momentum = params["momentum"]
sample_alpha = params["sample_alpha"]
bpreg = params["bpreg"]
logq = params["logq"]
hidden_act = params["hidden_act"]
n_epochs = 5
m = '1 5 10 20'

## Train & test the out-of-the-box model

In [7]:
train_script_oob, test_script_oob = create_gru4rec_tensorflow_script(model_name="gru4rec_tensorflow_oob", train_path=train_path, test_path=test_path, model_path=model_path, loss=loss, final_act=final_act, layers=layers, batch_size=50, dropout_p_hidden=dropout_p_hidden, learning_rate=learning_rate, hidden_act=hidden_act, n_epochs=n_epochs, m=m, decay=0.96, initial_accumulator=1e-1)

In [ ]:
from model_tracker import track_training_C02_emissions

track_training_C02_emissions(train_script_oob, "gru4rec_tensorflow_oob")

### Train the out-of-the-box model

In [10]:
os.system(train_script_oob)

0

### Test the out-of-the-box model

In [11]:
os.system(test_script_oob)

0

## Train & test the minor fix model


In [ ]:
train_script_minor, test_script_minor = create_gru4rec_tensorflow_script(model_name='gru4rec_tensorflow_minorfix', train_path=train_path, test_path=test_path, model_path=model_path, loss=loss, final_act=final_act, layers=layers, batch_size=batch_size, dropout_p_hidden=dropout_p_hidden, learning_rate=learning_rate, hidden_act=hidden_act, n_epochs=n_epochs, m=m, decay=1.0, initial_accumulator=1e-12)

### Train the minor fix model

In [ ]:
os.system(train_script_minor)

### Test the minor fix model

In [ ]:
os.system(test_script_minor)